In [1]:
import pandas as pd
from distributions import create_distribution

1. Create a classifier to predict the event (build a classifier)
2. What are the characteristics of each User Segment
3. Propose methods to get back active/dormant users

Unbalanced data!!!!!!! - how can we solve it?

In [2]:
input_data = pd.read_csv("data/CTM_DataScientistCaseStudy.csv", index_col=False)
print(input_data.shape)

(100000, 6)


In [47]:
input_data['Event'].value_counts()

0    70000
1    30000
Name: Event, dtype: int64

In [3]:
def create_age_groups(age):
    if age < 30:
        return '18-29'
    elif age < 45:
        return '30-45'
    elif age < 60:
        return '45-60'
    else:
        return '60+'

In [4]:
input_data['Age Group'] = input_data['Age'].apply(lambda x : create_age_groups(x))

In [5]:
print(input_data['Age'].min(), input_data['Age'].max())

18 75


In [6]:
input_data['UserSegment'].value_counts()

C    58517
A    21017
B    19973
Name: UserSegment, dtype: int64

In [7]:
featurelist =['Age Group','Recency']
for feature in featurelist:
    print(input_data.groupby(['UserSegment', feature])['UserID'].count())

UserSegment  Age Group
A            18-29         4779
             30-45         8306
             45-60         6074
             60+           1858
B            18-29         5224
             30-45         7772
             45-60         5237
             60+           1740
C            18-29        20415
             30-45        20091
             45-60        13111
             60+           4900
Name: UserID, dtype: int64
UserSegment  Recency 
A            Active      17770
             Dormant      1227
             Inactive     2020
B            Active      12298
             Dormant      3145
             Inactive     4530
C            Active      25360
             Dormant     17903
             Inactive    15254
Name: UserID, dtype: int64


In [8]:
input_data.groupby(['UserSegment', 'Age Group','Recency'])['UserID'].count()

UserSegment  Age Group  Recency 
A            18-29      Active      4247
                        Dormant      157
                        Inactive     375
             30-45      Active      7083
                        Dormant      474
                        Inactive     749
             45-60      Active      4977
                        Dormant      431
                        Inactive     666
             60+        Active      1463
                        Dormant      165
                        Inactive     230
B            18-29      Active      3687
                        Dormant      555
                        Inactive     982
             30-45      Active      4737
                        Dormant     1255
                        Inactive    1780
             45-60      Active      2990
                        Dormant      956
                        Inactive    1291
             60+        Active       884
                        Dormant      379
                        

In [13]:
input_data[input_data['UserSegment'].isnull()]

,UserID,Age,UserSegment,Recency,PriorEvent,Event,Age Group,weights
588,589,48,NaN,NaN,0,1,45-60,1
831,832,46,NaN,NaN,0,0,45-60,1
854,855,27,NaN,NaN,0,0,18-29,1
1050,1051,28,NaN,NaN,0,1,18-29,1
1453,1454,36,NaN,NaN,0,1,30-45,1
...,...,...,...,...,...,...,...,...
99790,99791,35,NaN,NaN,0,0,30-45,1
99846,99847,35,NaN,NaN,0,0,30-45,1
99873,99874,35,NaN,NaN,0,0,30-45,1
99938,99939,35,NaN,NaN,0,0,30-45,1


In [45]:
input_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 8 columns):
 #   Column       Non-Null Count   Dtype 
---  ------       --------------   ----- 
 0   UserID       100000 non-null  int64 
 1   Age          100000 non-null  int64 
 2   UserSegment  99507 non-null   object
 3   Recency      99507 non-null   object
 4   PriorEvent   100000 non-null  int64 
 5   Event        100000 non-null  int64 
 6   Age Group    100000 non-null  object
 7   weights      100000 non-null  int64 
dtypes: int64(5), object(3)
memory usage: 6.1+ MB


# Segments Profiling

In [40]:
def estimate_indexes(input_df:pd.DataFrame, target_variable:str, break_it_down_by:str, id_col:str= 'UserID'):
    distribution = pd.pivot_table(input_df, id_col, break_it_down_by, target_variable, 'count')
    distribution_percentages = distribution.div(distribution.sum(axis = 0))
    sample_distribution = input_df.groupby([break_it_down_by])[id_col].count()/user_segments_df.shape[0]
    distribution_indexes = distribution_percentages.div(sample_distribution, axis =0)

    return distribution_indexes

In [41]:
user_segments_df = input_data.dropna(subset = ['UserSegment'])
estimate_indexes(user_segments_df, 'UserSegment', 'Age Group', 'UserID')

UserSegment,A,B,C
Age Group,,,
18-29,0.743857,0.855624,1.141275
30-45,1.087272,1.070549,0.944575
45-60,1.177542,1.068345,0.912906
60+,1.035171,1.020100,0.980508


In [42]:
estimate_indexes(user_segments_df, 'UserSegment', 'Recency', 'UserID')

UserSegment,A,B,C
Recency,,,
Active,1.517893,1.105390,0.778022
Dormant,0.260801,0.703418,1.366720
Inactive,0.438630,1.035076,1.189650


In [43]:
estimate_indexes(user_segments_df, 'UserSegment', 'PriorEvent', 'UserID')

UserSegment,A,B,C
PriorEvent,,,
0,0.898341,1.007016,1.034117
1,2.000519,0.930950,0.664221
